# Sandbox to explore SEC N-PORT fund holdings information

## author:
- **David W. Hogg** (NYU)

## projects / notes:
- Trying to scrape SEC for data. This gets EVERYTHING for the quarter and then searches it.
- This downloads and reads a huge file! Probably we should reformat it into a `sqlite` file or something?
- Leaning heavily on `claude.ai` for help.

## bugs / to-do items:
- Works off of a union of search terms? That's not good.
- Hard-codes (doesn't figure out) the accession number. How to find this automagically?
- Finds and reads the data but does nothing with it.

In [1]:
# !pip install pandas

In [2]:
import os
import io
import requests
import hashlib
import zipfile
import pandas as pd

/Users/dhogg/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
# Change this to your name and email address; and adjust paths if you want.
HEADERS = {"User-Agent": "David W. Hogg dwhogg@gmail.com"}   # SEC requires this
DATA_DIR = os.path.expanduser("../data")
CACHE_DIR = os.path.expanduser("../data/cache")

In [4]:
# the story is: The fund name must contain (case-insensitive), ALL the keywords, not just ANY of them
# KEYWORDS = {"Vanguard", "social", }
# KEYWORDS = {"college", "social choice", }
KEYWORDS = {"college", "equity index", }
# KEYWORDS = {"Nuveen Large Cap", "Responsible", }
# KEYWORDS = {"Nuveen Large Cap", }
YEAR  = 2025
QUARTER = 3 # q1=Jan-Mar, q2=Apr-Jun, q3=Jul-Sep, q4=Oct-Dec

In [5]:
def get_with_cache(url, headers=None, timeout=180):
    # Use MD5 hex digest of URL as filename; last 2 chars as subdir
    digest = hashlib.md5(url.encode()).hexdigest()
    subdir = os.path.join(CACHE_DIR, digest[-2:])
    cache_path = os.path.join(subdir, digest)
    
    if os.path.exists(cache_path):
        print(f"Cache hit: {cache_path}")
        with open(cache_path, "rb") as f:
            return f.read()
    
    print(f"Downloading: {url}")
    r = requests.get(url, headers=headers, timeout=timeout)
    r.raise_for_status()
    print(f"Downloaded {len(r.content)/1e6:.1f} MB")
    
    os.makedirs(subdir, exist_ok=True)
    with open(cache_path, "wb") as f:
        f.write(r.content)
    
    return r.content

In [6]:
quarterstring = f"{YEAR}q{QUARTER}"
url = f"https://www.sec.gov/files/dera/data/form-n-port-data-sets/{quarterstring}_nport.zip"
r = get_with_cache(url, headers=HEADERS)

z = zipfile.ZipFile(io.BytesIO(r))
print("Files in zip:", z.namelist())

Cache hit: ../data/cache/70/bba26b0a3d72f9ec20fa516a5a970570
Files in zip: ['nport_metadata.json', 'nport_readme.htm', 'SUBMISSION.tsv', 'REGISTRANT.tsv', 'FUND_REPORTED_INFO.tsv', 'INTEREST_RATE_RISK.tsv', 'BORROWER.tsv', 'BORROW_AGGREGATE.tsv', 'MONTHLY_TOTAL_RETURN.tsv', 'MONTHLY_RETURN_CAT_INSTRUMENT.tsv', 'FUND_VAR_INFO.tsv', 'FUND_REPORTED_HOLDING.tsv', 'IDENTIFIERS.tsv', 'DEBT_SECURITY.tsv', 'DEBT_SECURITY_REF_INSTRUMENT.tsv', 'CONVERTIBLE_SECURITY_CURRENCY.tsv', 'REPURCHASE_AGREEMENT.tsv', 'REPURCHASE_COUNTERPARTY.tsv', 'REPURCHASE_COLLATERAL.tsv', 'DERIVATIVE_COUNTERPARTY.tsv', 'SWAPTION_OPTION_WARNT_DERIV.tsv', 'DESC_REF_INDEX_BASKET.tsv', 'DESC_REF_INDEX_COMPONENT.tsv', 'DESC_REF_OTHER.tsv', 'FUT_FWD_NONFOREIGNCUR_CONTRACT.tsv', 'FWD_FOREIGNCUR_CONTRACT_SWAP.tsv', 'NONFOREIGN_EXCHANGE_SWAP.tsv', 'FLOATING_RATE_RESET_TENOR.tsv', 'OTHER_DERIV.tsv', 'OTHER_DERIV_NOTIONAL_AMOUNT.tsv', 'SECURITIES_LENDING.tsv', 'EXPLANATORY_NOTE.tsv']


In [7]:
def read_tsv(z, name_fragment):
    candidates = [f for f in z.namelist() if name_fragment.upper() in f.upper()]
    if not candidates:
        raise FileNotFoundError(f"No file matching '{name_fragment}'. Available: {z.namelist()}")
    print(f"  Reading {candidates[0]} ...")
    with z.open(candidates[0]) as f:
        return pd.read_csv(f, sep="\t", dtype=str, low_memory=False)

sub      = read_tsv(z, "SUBMISSION")
reg      = read_tsv(z, "REGISTRANT")
fund     = read_tsv(z, "FUND_REPORTED_INFO")

print("SUBMISSION columns:", sub.columns.tolist())
print("REGISTRANT columns:",       reg.columns.tolist())
print("FUND_REPORTED_INFO columns:", fund.columns.tolist())

  Reading SUBMISSION.tsv ...
  Reading REGISTRANT.tsv ...
  Reading FUND_REPORTED_INFO.tsv ...
SUBMISSION columns: ['ACCESSION_NUMBER', 'FILING_DATE', 'FILE_NUM', 'SUB_TYPE', 'REPORT_ENDING_PERIOD', 'REPORT_DATE', 'IS_LAST_FILING']
REGISTRANT columns: ['ACCESSION_NUMBER', 'CIK', 'REGISTRANT_NAME', 'FILE_NUM', 'LEI', 'ADDRESS1', 'ADDRESS2', 'CITY', 'STATE', 'COUNTRY', 'ZIP', 'PHONE']
FUND_REPORTED_INFO columns: ['ACCESSION_NUMBER', 'SERIES_NAME', 'SERIES_ID', 'SERIES_LEI', 'TOTAL_ASSETS', 'TOTAL_LIABILITIES', 'NET_ASSETS', 'ASSETS_ATTRBT_TO_MISC_SECURITY', 'ASSETS_INVESTED', 'BORROWING_PAY_WITHIN_1YR', 'CTRLD_COMPANIES_PAY_WITHIN_1YR', 'OTHER_AFFILIA_PAY_WITHIN_1YR', 'OTHER_PAY_WITHIN_1YR', 'BORROWING_PAY_AFTER_1YR', 'CTRLD_COMPANIES_PAY_AFTER_1YR', 'OTHER_AFFILIA_PAY_AFTER_1YR', 'OTHER_PAY_AFTER_1YR', 'DELAYED_DELIVERY', 'STANDBY_COMMITMENT', 'LIQUIDATION_PREFERENCE', 'CASH_NOT_RPTD_IN_C_OR_D', 'CREDIT_SPREAD_3MON_INVEST', 'CREDIT_SPREAD_1YR_INVEST', 'CREDIT_SPREAD_5YR_INVEST', 'CREDIT

In [8]:
# Search all fund names in FUND_REPORTED_INFO for keywords
mask = fund["SERIES_NAME"].fillna("").str.upper().apply(
    lambda s: all(kw.upper() in s for kw in KEYWORDS)
)
Nmatch = mask.sum()
if Nmatch < 1:
    print("NO MATCHES FOUND (recall that we are and-ing the KEYWORDS")
    assert False
if Nmatch > 1:
    print("Uh-oh, found", Nmatch, "matches ...")
print(fund[mask][["ACCESSION_NUMBER", "SERIES_NAME"]].drop_duplicates().to_string())

           ACCESSION_NUMBER                                              SERIES_NAME
12089  0001752724-25-204872  College Retirement Equities Fund - Equity Index Account


In [9]:
# It is also possible to search registrant for related CIKs
# mask2 = reg["REGISTRANT_NAME"].str.upper().str.contains("VANGUARD", na=False)
# print(reg[mask2][["CIK", "REGISTRANT_NAME", "ACCESSION_NUMBER"]].drop_duplicates("CIK").to_string())

In [10]:
foo = fund[mask][["ACCESSION_NUMBER", "SERIES_NAME"]].iloc[0]
accession = foo["ACCESSION_NUMBER"]
series_name = foo["SERIES_NAME"]
print("Choosing (stupidly) to work with", accession, series_name)

Choosing (stupidly) to work with 0001752724-25-204872 College Retirement Equities Fund - Equity Index Account


In [11]:
frh = read_tsv(z, "FUND_REPORTED_HOLDING")
holdings = frh[frh["ACCESSION_NUMBER"] == accession].copy()
print("holdings rows:", len(holdings))
print("holdings columns:", list(holdings.keys()))

  Reading FUND_REPORTED_HOLDING.tsv ...
holdings rows: 522
holdings columns: ['ACCESSION_NUMBER', 'HOLDING_ID', 'ISSUER_NAME', 'ISSUER_LEI', 'ISSUER_TITLE', 'ISSUER_CUSIP', 'BALANCE', 'UNIT', 'OTHER_UNIT_DESC', 'CURRENCY_CODE', 'CURRENCY_VALUE', 'EXCHANGE_RATE', 'PERCENTAGE', 'PAYOFF_PROFILE', 'ASSET_CAT', 'OTHER_ASSET', 'ISSUER_TYPE', 'OTHER_ISSUER', 'INVESTMENT_COUNTRY', 'IS_RESTRICTED_SECURITY', 'FAIR_VALUE_LEVEL', 'DERIVATIVE_CAT']


In [12]:
# weird issues with the PERCENTAGE column
holdings["PERCENTAGE"] = holdings["PERCENTAGE"].astype(float)
print(holdings.nlargest(10, "PERCENTAGE")[["ISSUER_NAME", "PERCENTAGE"]])

                    ISSUER_NAME  PERCENTAGE
3901105             NVIDIA Corp    7.301205
3908536          Microsoft Corp    7.005696
3953574               Apple Inc    5.806902
3931309          Amazon.com Inc    3.928106
3901087      Meta Platforms Inc    3.036674
3871096            Broadcom Inc    2.456034
3901086            Alphabet Inc    1.942845
3908732  Berkshire Hathaway Inc    1.687943
3975978               Tesla Inc    1.686808
3931308            Alphabet Inc    1.578102


In [13]:
print("total of all percentages:", holdings["PERCENTAGE"].sum())

total of all percentages: 100.51424473723


In [14]:
# write out a CSV file
filename = os.path.join(DATA_DIR, series_name.replace(" ", "_") + f"_{quarterstring}.csv")
holdings.to_csv(filename, index=False)
print("wrote", filename)

wrote ../data/College_Retirement_Equities_Fund_-_Equity_Index_Account_2025q3.csv
